# Introduction

# Imports

In [ ]:
from pathlib import Path
from harbor.plotting.ligands import plot_aligned_ligands

In order to avoiding having this repo depend directly on the asapdiscovery repo, I'm going to comment this out, but we'll use a test example from the covid moonshot molecules:
```
from asapdiscovery.data.testing.test_resources import fetch_test_file
mypath = fetch_test_file("Mpro_combined_labeled.sdf")
``` 

In [ ]:
mypath = Path("../data/Mpro_combined_labeled.sdf")

## use p-series curated subset

In [ ]:
mypath = Path("../data/combined_3d.sdf")

I'm copying this code from the asapdiscovery repo.
Once that is conda installable, I'll make that a dep of this repo and use those tools for loading molecules

# Load Molecules

In [ ]:
from rdkit import Chem
mols = Chem.SDMolSupplier(str(mypath))

In [ ]:
mols = [mol for mol in mols]

In [ ]:
import mols2grid

In [ ]:
# define the grid to show the scaffolds
grid = mols2grid.display(mols)

In [ ]:
grid

# MCSS-based Clustering

In [ ]:
from harbor.clustering.hierarchical import ClusterResults, ClusterCenter, HeirarchicalClustering
from openeye import oechem

In [ ]:
mol: Chem.Mol = mols[0]
mol.GetPropsAsDict()

In [ ]:
oemols = []
mol_ids = []
for rdkit_mol in mols:
    smiles = Chem.MolToSmiles(rdkit_mol)
    properties = rdkit_mol.GetPropsAsDict()
    mol_ids.append(properties["compound_name"])
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smiles)
    oemols.append(mol)

In [ ]:
from harbor.clustering import hierarchical as h
from importlib import reload
reload(h)

In [ ]:
clusterer = h.HeirarchicalClustering(molecules=oemols, mol_ids=mol_ids)

In [ ]:
clusterer

In [ ]:
clusters = clusterer.cluster(max_iterations=50, cutoff=15)

In [ ]:
def get_descendents(cluster):
    descendents = []
    for child in cluster.children:
        if isinstance(child, str):
            descendents.append(cluster)
        else:
            descendents.extend(get_descendents(child))
    return descendents

In [ ]:
from harbor.plotting import ligands as l
reload(l)

In [ ]:
clusters.keys()

In [ ]:
len(mol_ids)

In [ ]:
len(set(mol_ids))

In [ ]:
children = {desc.cluster_id: desc for cluster_id, cluster in clusters.items() for desc in cluster.children}

In [ ]:
grandchildren = {desc.cluster_id: desc for cluster_id, cluster in children.items() for desc in cluster.children}

In [ ]:
greatgrandchildren= {desc.cluster_id: desc for cluster_id, cluster in grandchildren.items() for desc in cluster.children}

In [ ]:
greatgreatgrandchildren = {desc.cluster_id: desc for cluster_id, cluster in greatgrandchildren.items() for desc in cluster.children if not cluster.height == 0}

In [ ]:
l5 = {desc.cluster_id: desc for cluster_id, cluster in greatgreatgrandchildren.items() for desc in cluster.children if not cluster.height == 0}

In [ ]:
current_level_clusters = {desc.cluster_id: desc for cluster_id, cluster in l5.items() for desc in cluster.children if not cluster.height == 0}

In [ ]:
for i in range(6, 19):
    current_level_clusters = {desc.cluster_id: desc for cluster_id, cluster in current_level_clusters.items() for desc in cluster.children if not cluster.height == 0}
    ids_found = []
    for cluster_id, cluster in l6.items():
        print(f"Cluster {cluster_id}")
        descendents, _, _ = get_descendents(cluster)
        print(f"Children: {len(descendents)}")
        mols = []
        for desc in descendents:
            mol = desc.repr
            mol.SetTitle(desc.children[0])
            mols.append(mol)
        l.plot_ligands_with_mcs(filename=f"cluster_{cluster_id}.png", mols=mols, mcs_mol=cluster.repr, reference="largest")
        ids_found.extend([desc.children[0] for desc in descendents])

In [ ]:
total_mols_found = []
for cluster_id, cluster in greatgreatgrandchildren.items():
    total_mols_found.extend([desc.children[0] for desc in get_descendents(cluster)])

In [ ]:
greatgreatgrandchildren.keys()

In [ ]:
len(total_mols_found)

In [ ]:
len(set(total_mols_found))

In [ ]:
set(ids_found)

In [ ]:
set(mol_ids) - set(ids_found)

In [ ]:
# Try building a graph

In [ ]:
def get_descendents(cluster):
    descendents = []
    full_cluster_dict = {}
    pairs = []
    heights = []

    if cluster.height > 0:
        pairs.append([child.cluster_id for child in cluster.children])
        heights.append(cluster.height)
    
    for child in cluster.children:

        if isinstance(child, str):
            descendents.append(cluster)
            full_cluster_dict[child] = cluster
        else:
            childrens_descendents, childrens_full_cluster_dict, childrens_pairs, childrens_heights = get_descendents(child)
            descendents.extend(childrens_descendents)
            full_cluster_dict[child.cluster_id] = child
            full_cluster_dict.update(childrens_full_cluster_dict)
            pairs.extend(childrens_pairs)
            heights.extend(childrens_heights)
    return descendents, full_cluster_dict, pairs, heights

In [ ]:
desc, id_dict, pairs, heights = get_descendents(clusters['19_0'])

In [ ]:
clusters['19_0'].height

In [ ]:
len(desc)

In [ ]:
len(set(id_list))

In [ ]:
len(id_list)

In [ ]:
len(clusters)

In [ ]:
clusters.keys()

In [ ]:
import json

In [ ]:
with open("id_dict.json", 'w') as f:
    json.dump(list(id_dict.keys()), f)

In [ ]:
with open("pairs.json", 'w') as f:
    json.dump(pairs, f)

In [ ]:
import numpy as np
sorted_cluster_ids = np.sort(np.array(list(id_dict.keys())))

# sort pairs

In [ ]:
pair_array = np.array(pairs)

In [ ]:
heights_array = np.array(heights)

## sort the pairs by the heights

In [ ]:
sorted_pair_array = pair_array[np.argsort(heights_array)]

In [ ]:
merges = [(int(np.where(sorted_cluster_ids == i)[0][0]), int(np.where(sorted_cluster_ids == j)[0][0])) for i, j in sorted_pair_array]

In [ ]:
with open("merges.json", 'w') as f:
    json.dump(merges, f)